## 입력한 상품을 취급하는 매장 3곳 찾기

In [83]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [104]:
# 상품명
title = '무'

In [105]:
# 비교 데이터프레임 호출
list_df = pd.read_csv('../data/eco_product.csv')
list_df.head(3)

,title,point,market,title_mod
0,수실류-은행,10,로컬푸드 직매장,수실류은행
1,수실류-호두,10,로컬푸드 직매장,수실류호두
2,수실류-대추,10,로컬푸드 직매장,수실류대추


In [106]:
# 찾는 품목이 '무' 또는 '김'처럼 한 글자일 경우
# TfidfVectorizer의 Default 값으로 한 글자 단어는 제외하게 되어있으므로
# 하이퍼 파라미터로 해결 23.10.25

title = [title.strip(' ')]

# 찾는 품목이 한 글자 단어일 경우
if len(title[0]) == 1:
    # TF-IDF 벡터화
    tv = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}')
    title_tv = tv.fit_transform(title).toarray()
# 찾는 품목이 두 글자 이상일 경우
else:
    # 데이터프레임 화
    title_df = pd.DataFrame(title, columns=['title'])
    # TF-IDF 벡터화
    tv = TfidfVectorizer()
    title_tv = tv.fit_transform(title_df['title'])

# 품목 리스트 TF-IDF 벡터화
list_tv = tv.transform(list_df['title'])

In [107]:
# 코사인 유사도 계산
cosine_similarities = cosine_similarity(title_tv, list_tv)
cosine_similarities

array([[0., 0., 0., ..., 0., 0., 0.]])

In [108]:
# 유사도 컬럼으로 추가
list_df['cosine_similarity'] = cosine_similarities.reshape(-1,1)

In [109]:
# cosine_similarity의 값이 1.0인 경우만 추출
list_df = list_df[list_df['cosine_similarity'] == 1.0]
list_df

,title,point,market,title_mod,cosine_similarity
53,무(1kg/무농약/온라인),120,두레생협,무,1.0
57,무(김장/5kg/무농약),350,두레생협,무,1.0
186,친환경 무(개),170,이마트,친환경 무,1.0
1780,[친환경]무(친환경/2개/1.4kg이상),140,올가홀푸드,무,1.0
2491,근채류-무,1,로컬푸드 직매장,근채류무,1.0
2573,[김장]무(무농약1개/1.2kg내외),120,올가홀푸드,무,1.0
2899,무(친환경/무)1개/랩/특/특품,70,하나로마트,무개랩특특품,1.0
3865,무(1.5kg이상/무농약),130,두레생협,무,1.0


In [110]:
# 관련된 마켓명
global market
market = list(set(list_df['market'].values))[:3]
if market == []:
    print('해당 상품을 취급하는 매장이 없습니다.')
market

['두레생협', '이마트', '하나로마트']